In [1]:
import re
import time
import json
import pickle
import datetime

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [6]:
driver = webdriver.Chrome('chromedriver.exe')
driver.get('https://www.oddsportal.com/esports/results/')

C:\Users\royta\AppData\Local\Temp\ipykernel_12424\2489293918.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


In [7]:
def wait(className):
    max_wait_num = 4
    for _ in range(max_wait_num):
        try:
            myElem = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME, className)))
            break
        except TimeoutException:
            if _ >= max_wait_num-1:
                return None

In [8]:
def scarp_games(time_point:int):
    games_links = []
    soup = BeautifulSoup(driver.page_source, 'html')
    table = soup.find('table', attrs={"id": 'tournamentTable'})
    for game in table.find_all('tr', class_ = "odd deactivate"):
        games_links.append(game.find('a').get('href'))

    next_page = soup.find('span', class_ = "arrow", text ='»')
    while next_page:
        time.sleep(1)
        driver.get(tournament + next_page.find_parent().get('href'))
        wait("col-content")

        soup = BeautifulSoup(driver.page_source, 'html')
        table = soup.find('table', attrs={"id": 'tournamentTable'})
        for game in table.find_all('tr', class_ = "odd deactivate"):
            games_links.append(game.find('a').get('href'))

        next_page = soup.find('span', class_ = "arrow", text ='»')
        link = next_page.find_parent().get('href')
        obj = re.search(r"#/page/\d+/", link)
        num1 = int(re.search(r"\d+", obj[0])[0])
        num2 = int(next_page.find_parent().get_attribute_list('x-page')[0])
        if num1 != num2:
            break
        
    games = []
    for link in games_links:
        try:
            driver.get(f"https://www.oddsportal.com{link}")
            wait("col-content")
            soup = BeautifulSoup(driver.page_source, 'html')
            working_space = soup.find("div", attrs = {"id":"col-content"})

            teams_names = working_space.find('h1').text
            date = working_space.find('p').text
            
            time_stamp = int(time.mktime(datetime.datetime.strptime(date, '%A, %d %b %Y, %H:%M').timetuple()))
            
            if time_stamp >= time_point:
                score = working_space.find("div", attrs = {"id":"event-status"}).find('p').find('strong').text
                odds = working_space.find("tr", class_="aver").find_all('td', class_="right")
                if len(odds) == 3:
                    t1_odd = working_space.find("tr", class_="aver").find_all('td', class_="right")[0].text
                    x_odd = working_space.find("tr", class_="aver").find_all('td', class_="right")[1].text
                    t2_odd = working_space.find("tr", class_="aver").find_all('td', class_="right")[2].text
                    games.append((teams_names, date, score, (t1_odd, x_odd, t2_odd), link))
                else:
                    t1_odd = working_space.find("tr", class_="aver").find_all('td', class_="right")[0].text
                    t2_odd = working_space.find("tr", class_="aver").find_all('td', class_="right")[1].text
                    games.append((teams_names, date, score, (t1_odd, t2_odd), link))
                
        except Exception as e:
            print(e)
            
        
    return games

In [9]:
with open('tournaments.json', "r") as j:
    tournaments = json.load(j)
    
games = []
months = 8
time_point = time.time() - 60*60*24*30*months
for tournament in tournaments:
    driver.get(tournament)
    wait("col-content")
    games += scarp_games(time_point)
    
    with open('odds.json', 'w') as f:
        json.dump(games, f)

'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
time data 'Today, 12 Aug  2022, 15:00' does not match format '%A, %d %b %Y, %H:%M'
time data 'Yesterday, 11 Aug  2022, 22:35' does not match format '%A, %d %b %Y, %H:%M'
time data 'Yesterday, 11 Aug  2022, 15:00' does not match format '%A, %d %b %Y, %H:%M'
'NoneType' object has no attribute 'find'
Message: unknown error: net::ERR_CONNECTION_TIMED_OUT
  (Session info: chrome=104.0.5112.81)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B178B3+2193587]
	Ordinal0 [0x00AB0681+1771137]
	Ordinal0 [0x009C41A8+803240]
	Ordinal0 [0x009C0918+788760]
	Ordinal0 [0x009B781D+751645]
	Ordinal0 [0x009B8450+754768]
	Ordinal0 [0x009B7A2A+752170]
	Ordinal0 [0x009B6FAF+749487]
	Ordinal0 [0x009B5E94+745108]
	Ordinal0 [0x009B63